# Neptune + Prophet

## Introduction

This quickstart will show you how to (using just single function) log FBprophet:

* Regression summary metadata,
* Input and forecast Dataframes,
* Forecast, components and residual diagnostics plots,
* Model configuration,
* Serialized model

## Before you start

This notebook example lets you try out Neptune as an anonymous user, with zero setup.

* If you are running the notebook on your local machine, you need to have [Python](https://www.python.org/downloads/) and [pip](https://pypi.org/project/pip/) installed.
* If you want to see the example recorded to your own workspace instead:
    * Create a Neptune account → [Take me to registration](https://neptune.ai/register)
    * Create a Neptune project that you will use for tracking metadata → [Tell me more about projects](https://docs.neptune.ai/administration/projects)

## Install Neptune and dependencies

In [ ]:
! pip install neptune-client neptune-prophet==1.0.0

In [9]:
import neptune.new as neptune

## Initialize Neptune and create new run
Connect your script to Neptune application and create new run.


In [3]:
run = neptune.init(
    project='common/fbprophet-integration', 
    api_token="ANONYMOUS",
    tags=["fbprophet", "artifacts", "notebook"]    
)

Info (NVML): NVML Shared Library Not Found. GPU usage metrics may not be reported. For more information, see https://docs-legacy.neptune.ai/logging-and-managing-experiment-results/logging-experiment-data.html#hardware-consumption 


https://app.neptune.ai/common/fbprophet-integration/e/FBPROP-19
Remember to stop your run once you’ve finished logging your metadata (https://docs.neptune.ai/api-reference/run#stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.


Click on the link above to open this run in Neptune. For now it is empty but keep the tab with run open to see what happens next.

You tell Neptune: 

* **who you are**: your Neptune API token `api_token` 
* **where you want to send your data**: your Neptune `project`.

At this point you have new run in Neptune. For now on you will use `run` to log metadata to it.

---

**Note**


Instead of logging data to the public project `'common/sklearn-integration'` as an anonymous user 'neptuner' you can log it to your own project.

To do that:

1. Get your [Neptune API token](https://docs.neptune.ai/getting-started/installation#get-api-token)
2. Pass the token to ``api_token`` argument of ``neptune.init()`` method: ``api_token=YOUR_API_TOKEN``
3. Pass your project to the ``project`` argument of ``neptune.init()``.

For example:

```python
neptune.init(project="YOUR_WORKSPACE/YOUR_PROJECT",
             api_token="YOUR_API_TOKEN")
```

In [ ]:

import pandas as pd

In [11]:
df = pd.read_csv("https://raw.githubusercontent.com/facebook/prophet/master/examples/example_wp_log_R.csv")
df["cap"] = 8.5

In [ ]:
from prophet import Prophet

In [23]:

model = Prophet(growth='logistic')
model.fit(df)

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Initial log joint probability = -19.9808


    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       9006.46      0.012704       469.696           1           1      113   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199       9014.75     0.0122174       453.437       2.277      0.2277      232   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     299       9018.94   0.000458159       104.683      0.6529      0.6529      362   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     399       9019.52   9.46772e-05       91.5897           1           1      498   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     421       9019.56   6.88623e-05        72.725   9.252e-07       0.001      569  LS failed, Hessian reset 
     444       9019.56   2.64826e-07       72.1751     0.03109      0.8902      6

In [35]:
future = model.make_future_dataframe(periods=365*5)
future["cap"] = 8.5

forecast = model.predict(future)
fig = model.plot(forecast)

### Log summary metadata

In [ ]:
import neptune_prophet.impl as npt_utils

run["prophet_summary"] = npt_utils.create_summary(model=model, df=df, fcst=forecast)

You just learned how to log FBProphet regression summary to Neptune using single function.

Click on the link that was outputted to the console or [go here](https://app.neptune.ai/o/common/org/fbprophet-integration/e/FBPROP-249/all?path=prophet_summary%2Fdiagnostics_charts%2F) to explore a run similar to yours. In particular check:

* [logged dataframes](https://app.neptune.ai/o/common/org/fbprophet-integration/e/FBPROP-249/all?path=prophet_summary%2Fdataframes%2F)
* [logged regressor visualizations](https://app.neptune.ai/o/common/org/fbprophet-integration/e/FBPROP-249/all?path=prophet_summary%2Fdiagnostics_charts%2F)
* [logged model config](https://app.neptune.ai/o/common/org/fbprophet-integration/e/FBPROP-249/all?path=prophet_summary%2Fmodel%2Fmodel_config%2F),
* [logged serialized model](https://app.neptune.ai/o/common/org/fbprophet-integration/e/FBPROP-249/all?path=prophet_summary%2Fmodel%2F&attribute=serialized_model),
* [logged metadata](https://app.neptune.ai/o/common/org/fbprophet-integration/e/FBPROP-249/all?path=sys),
* [logged code and git metadata](https://app.neptune.ai/o/common/org/fbprophet-integration/e/FBPROP-249/source-code?path=source_code&file=Neptune_prophet.py).

## More Options

### Log FBprophet plots

In [ ]:
run["forecast_plots"] = npt_utils.create_forecast_plots(model, forecast)

In [ ]:
run["forecast_components"] = npt_utils.get_forecast_components(model, forecast)

In [ ]:
run["residual_diagnostics_plot"] = npt_utils.create_residual_diagnostics_plots(forecast, df.y)

### Log FBprophet model configuration 

In [ ]:
run["model_config"] = npt_utils.get_model_config(model)

### Log FBprophet serialized model

In [1]:
run["model"] = npt_utils.get_serialized_model(model)

Exception: File `'[model].py'` not found.

## Stop run

<font color=red>**Warning:**</font><br>
Once you are done logging, you should stop tracking the run using the `stop()` method.
This is needed only while logging from a notebook environment. While logging through a script, Neptune automatically stops tracking once the script has completed execution.

In [20]:
run.stop()

Shutting down background jobs, please wait a moment...
Done!


Waiting for the remaining 11 operations to synchronize with Neptune. Do not kill this process.


All 11 operations synced, thanks for waiting!
